In [84]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import os


import statsmodels.api as sm
import statsmodels
import lightgbm as lgb

main_directory = Path.cwd().parent.as_posix()

In [85]:
df = pd.read_csv(f"{main_directory}/databases/zoopla_data.csv")
df = pd.get_dummies(df, columns = ['type'], drop_first = True, prefix = 'type')
df = pd.get_dummies(df, columns = ['location'], drop_first = True, prefix = 'location')
df

,price,station,beds,baths,receptions,post_code,avg_sold_price_12months,type_end terrace,type_flat,type_semi-detached,...,location_cheshire,location_hyde,location_lancashire,location_manchester,location_oldham,location_rochdale,location_salford,location_stalybridge,location_stockport,location_wigan
0,369950.0,0.5,1,1.0,0.0,M15,200730.0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
1,369950.0,0.5,1,1.0,0.0,M15,200730.0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
2,154950.0,0.5,2,1.0,1.0,M15,200730.0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
3,541805.0,0.2,3,1.0,1.0,M15,200730.0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
4,240327.0,0.2,1,1.0,1.0,M15,200730.0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4371,625000.0,0.7,3,2.0,3.0,M34,193433.0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4372,230000.0,0.7,3,1.0,2.0,M34,193433.0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
4373,310000.0,0.6,4,1.0,0.0,M34,193433.0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
4374,130000.0,0.2,3,1.0,1.0,M34,193433.0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [86]:
x = df.drop(['price','post_code'],axis=1)
y = df.price

In [87]:
from sklearn.model_selection import train_test_split
from sklearn import metrics

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)



In [88]:
model = lgb.LGBMRegressor(learning_rate=0.03,max_depth=2,random_state=42,num_iterations=50000)
model.fit(x_train,y_train,eval_set=[(x_test,y_test),(x_train,y_train)],eval_metric='RMSE',verbose=50,early_stopping_rounds=100)

/home/tymo/Desktop/ZooplaScrape/datascience/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/tymo/Desktop/ZooplaScrape/datascience/lib/python3.9/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/home/tymo/Desktop/ZooplaScrape/datascience/lib/python3.9/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead

[50]	training's rmse: 104898	training's l2: 1.10036e+10	valid_0's rmse: 123320	valid_0's l2: 1.52078e+10
[100]	training's rmse: 97009.7	training's l2: 9.41088e+09	valid_0's rmse: 115215	valid_0's l2: 1.32744e+10
[150]	training's rmse: 93449.8	training's l2: 8.73286e+09	valid_0's rmse: 111392	valid_0's l2: 1.24081e+10
[200]	training's rmse: 91020.7	training's l2: 8.28476e+09	valid_0's rmse: 109147	valid_0's l2: 1.1913e+10
[250]	training's rmse: 89487.1	training's l2: 8.00793e+09	valid_0's rmse: 107769	valid_0's l2: 1.16141e+10
[300]	training's rmse: 87852.3	training's l2: 7.71802e+09	valid_0's rmse: 106238	valid_0's l2: 1.12864e+10
[350]	training's rmse: 86813.4	training's l2: 7.53657e+09	valid_0's rmse: 105217	valid_0's l2: 1.10706e+10
[400]	training's rmse: 86041.9	training's l2: 7.4032e+09	valid_0's rmse: 104470	valid_0's l2: 1.09139e+10
[450]	training's rmse: 85424.7	training's l2: 7.29739e+09	valid_0's rmse: 103904	valid_0's l2: 1.0796e+10
[500]	training's rmse: 84850.7	training's 

LGBMRegressor(learning_rate=0.03, max_depth=2, num_iterations=50000,
              random_state=42)